# magics

> Fill in a module description here

In [ ]:
#| default_exp magics

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import os
import openai
from IPython.display import display, Markdown

In [ ]:
#| export
#| notest
openai.api_key = os.environ['OPENAI_API_KEY']
if openai.api_type == "azure":
    deployment_name = os.environ['OPENAI_DEPLOYMENT_NAME']
CONTEXT_MAX_WORDS = 2200

In [ ]:
#| export
#| notest

from abc import ABC, abstractmethod

class OpenAIAPI(ABC):
    def __init__(self):
        self.reset_context()

    @abstractmethod
    def reset_context(self):
        pass

    def get_completion(self, prompt, new_conversation=True):
        if new_conversation:
            self.reset_context()
        
        self.context.append(
            {
                'role':'user',
                'content': prompt
            }
        )

        if openai.api_type == "azure":
            response = openai.ChatCompletion.create(
                model = deployment_name,
                messages = self.context
            )
        else:
            response = openai.ChatCompletion.create(
                model = "gpt-3.5-turbo",
                messages = self.context
            )
        
        completion = self.extract_completion(response)
        self.extend_context(response)
        self.prune_context()

        self.completion = completion    
        self.response = response  # useful for debugging
    
    def extract_completion(self, response):
        return response['choices'][0].message.content.strip()
    
    def extend_context(self, response):
        self.context.append(response['choices'][0].message.to_dict())
    
    def prune_context(self):
        # Prune context to under CONTEXT_MAX_WORDS words. That should be ~CONTEXT_MAX_WORDS*1.5 tokens, leaving room for the prompt and completion.
        pruned_context = []
        word_count = 0
        while self.context:
            last_message = self.context.pop()
            word_count += len(last_message['content'].split())
            if word_count < CONTEXT_MAX_WORDS:
                pruned_context.append(last_message)
            else:
                break
        pruned_context.reverse()
        self.context = pruned_context

    def display_completion(self):
        display(Markdown(self.completion))

class ConversationAPI(OpenAIAPI):
    def reset_context(self):
        self.context = [
            {
                'role': 'system',
                'content': 'You are an expert programmer helping out a friend. Your friend is using Python in Jupyter Notebook. Give a succinct answer that a programmer with one year of professional experience would easily understand.'
            }
        ]

class CodingAPI(OpenAIAPI):
    def reset_context(self):
        self.context = [
            {
                'role': 'system',
                'content':
                    '''
                        You are a programming assistant. You will be passed code and instruction what to do next. Output the code that should be added next. Your prompt will be in the following format:

                        Code: {code}
                        Instruction: {instruction}

                        Output only the code that should be added next. Do not output the entire code. Do not output the instruction. Do not output the prompt. Do not output any other text. Do not output any lines that are not indented correctly. Do not output any lines that are not valid Python.
                    '''
            }
        ]

conversation_api = ConversationAPI()
coding_api = CodingAPI()

In [ ]:
#| export
def collect_code_history():
    history = [cell_content for session, cell_number, cell_content in get_ipython().history_manager.get_tail()]
    collected_code = ''
    word_count = 0
    while history:
        last_cell_content = history.pop()
        word_count += len(last_cell_content.split())
        if word_count < CONTEXT_MAX_WORDS:
            collected_code += ' ' + last_cell_content
        else:
            break
    return collected_code

In [ ]:
#| export

import base64
import re

def ai_ask(line, cell):
    conversation_api.get_completion(cell)
    conversation_api.display_completion()

def ai_continue(line, cell):
    conversation_api.get_completion(cell, False)
    conversation_api.display_completion()
    
def ai_code(line, cell):
    prompt = f'Code: {collect_code_history()}\nInstruction: {cell}'
    coding_api.get_completion(prompt)
    
    encoded_code = base64.b64encode(coding_api.completion.encode()).decode()
    js_code = f"""
        var new_cell = Jupyter.notebook.insert_cell_below('code');
        new_cell.set_text(atob("{encoded_code}"));
    """
    get_ipython().run_cell_magic('javascript', '', js_code)

In [ ]:
#| export

def load_ipython_extension(ipython):
    ipython.register_magic_function(ai_ask, magic_kind='cell', magic_name='ai_ask')
    ipython.register_magic_function(ai_continue, magic_kind='cell', magic_name='ai_continue')
    ipython.register_magic_function(ai_code, magic_kind='cell', magic_name='ai_code')

In [ ]:
load_ipython_extension(get_ipython())

In [ ]:
#| notest
%%ai_ask

How to write an abstract clsss in Python?

In Python, to write an abstract class, you need to use the `ABC` (Abstract Base Class) module from the `abc` library. You can define an abstract method by decorating the method with the `@abstractmethod` decorator. Here's an example:

```
from abc import ABC, abstractmethod

class MyAbstractClass(ABC):
    
    @abstractmethod
    def my_method(self):
        pass
```

In this example, `MyAbstractClass` is an abstract class that defines an abstract method `my_method()`. Any subclass inheriting from this abstract class must implement `my_method()` or it will result in an error.

In [ ]:
#| notest
%%ai_continue

Can you give me an example?

Sure!

Let's say you want to create a program that performs calculations on different geometric shapes like rectangle, circle, and triangle. All these shapes share some common attributes like area and perimeter.

To create an abstract class that defines these attributes as abstract methods, you can use the following code:

```python
from abc import ABC, abstractmethod

class Shape(ABC):
    @abstractmethod
    def area(self):
        pass

    @abstractmethod
    def perimeter(self):
        pass
```

In this example, `Shape` is an abstract base class that defines two abstract methods, `area` and `perimeter`. Any class that inherits from this `Shape` class must implement these two methods or else it will result in an error.

Now, let's say you want to create a `Rectangle` class that inherits from the `Shape` abstract class:

```python
class Rectangle(Shape):
    def __init__(self, length, width):
        self.length = length
        self.width = width

    def area(self):
        return self.length * self.width

    def perimeter(self):
        return 2 * (self.length + self.width)
```

In this example, `Rectangle` is a concrete class that implements the abstract methods `area` and `perimeter`. It defines the `__init__` method to initialize the length and width of the rectangle. The `area` method calculates the area of the rectangle, and the `perimeter` method calculates the perimeter of the rectangle.

You can create other classes like `Circle` and `Triangle` and inherit from the `Shape` class to implement the `area` and `perimeter` methods according to the properties of the specific shape.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()